# DQN 구현하기

## 필요한 모듈 import

In [2]:
import tensorflow.compat.v1 as tf
import numpy as np
import random

from collections import deque

## 하이퍼파라미터

### REPLAY_MEMORY
- 학습에 사용할 플레이 결과를 얼마나 많이 저장해서 사용할 지를 정함

### BATCH_SIZE
- 한번 학습할 때 몇개의 기억을 사용할지를 정함
- 미니배치의 크기

### GAMMA
- 오래된 상태의 가중치를 줄이기 위한 하이퍼파라미터

### STATE_LEN
- 한 번에 볼 프레임의 총 수
- 에이전트에서 DQN에 상태를 넘겨줄 때는 한 프레임의 상태만 넘겨줌
- 다만, DQN이 상태를 받으면 해당 상태만이 아니라 STATE_LEN - 1 개의 앞 상태까지 고려해서 현재의 상태로 만들어 저장
- 즉, 이전 상태까지 고려하는 것

## DQN 객체 초기화
- DQN 객체를 초기화
- 텐서플로 세션과 가로/세로 크기, 그리고 행동의 개수를 받아 객체를 초기화

### self.memory = deque()
- 게임 플레이 결과를 저장할 메모리!
- collections ahebfdml deque() 함수로 만들어진 객체는 배열과 비슷, 맨 처음 들어간 요소 쉽게 제거 *popleft*
- 저장할 기억의 개수를 제한!

## 플레이스 홀더 설정
### input_X
- 게임의 상태를 입력
- 구조: [게임판 가로 크기, 게임판 세로 크기, 게임 상태의 개수(현재+과거+과거+...)] 형식

### input_A
- 각 상태를 만들어낸 액션의 값을 입력 받음
- 원-핫 인코딩 아닌 행동을 나타내는 수자를 그대로 받음

### input_Y
- 손실값 계산에 사용할 값을 입력 받음
- 손실값 = (보상 + 목표 신경망(target)으로 구한 다음 상태의 Q값) - 학습 신경망에서 구한 Q 값 으로 하여 학습 진행

### Q 값
- 행동에 따른 가치를 나타내는 값
- **목표 신경망에서 구한 Q 값**: 구한 값 중 최대의 값(최적 행동) 사용
- **학습 신경망에서 구한 Q 값**: 현재 행동에 따른 값을 사용
- 행동 선택시 가장 가치가 높은 행동을 선택하도록 학습할 것
- 뒤에 train 함수와 _build_op 함수의 Q_value 값들을 참조

## 신경망 구성
- 학습을 진행할 신경망과 목표 신경망 구성
- 두 신경망은 구성이 같음 -> 신경망을 구성하는 함수는 같은 것을 사용! 이름만 다름
- 목표 신경망은 단순히 Q 값을 구하는 데만 사용 (손실값과 최적화 함수를 사용하지 않음)
### NOTE
- 학습 신경망
    - 학습을 진행할 때마다 "가중치들이 갱신"되므로 학습 신경망이라고 함
    - 실제 게임 진행할때 행동 예측하는 데 사용하는 주 신경망
    
    
- 목표 신경망
    - 학습 시에만 보조적으로 사용하는 신경망

## 신경망 구성 함수 정의 _build_network
- 학습 신경망과 목표 신경망 구성 함수
- 상태값 input_X -> 행동의 가짓수만큼의 출력값 생성
- 위 값들의 최댓값을 취함 = 다음 행동  


- 간단한 CNN 으로 되어 있다.
- 특이한 점: 풀링 계층이 없다; 표현력을 높여 이미지의 세세한 부분까지 판단하기 위해
- 게임 환경에 따라 CNN의 필터 크기나 층수를 조절해야 좋은결과를 얻는다.

## DQN 손실 함수 정의
- 현재 상태를 이용해 학습 신경망으로 구한 Q_value와 다음상태를 이용해 목표 신경망으로 구한 Q_value(input_Y)를 이용하여 손실값을 구하고 최적화

### tf.multiply(self.Q, one_hot) 
- self.Q 로 구한 값에서 현재 행동의 인덱스에 해당하는 값만 선택하기 위해 사용

### one_hot
- 현재 행동의 인덱스에 해당하는 값에만 1, 나머지에는 0이 들어있음
- Q 값과 one_hot 값을 곱하면 현재 행동의 인덱스에 해당하는 값만 남고 전부 0 이됨

## 목표 신경망 갱신 함수
- 학습 신경망의 변수들의 값을 목표 신경망으로 복사해서 목표 신경망의 변수들을 최신 값으로 갱신하는 것

## 다음 행동 정의 함수
### get_action
- 현재 상태를 이용해 다음에 취해야 할 행동을 찾는 함수
- _build_network 함수에서 계산한 Q_value를 이용
- 출력값이 원-핫 인코딩 되어 있음 -> np.argmax 함수를 이용해 최댓값 담긴 index 값을 행동값으로 취함..
- 학습에 필요한 텐서와 연산이 모두 준비됨

### 요약
- CNN 을 사용한 신경망으로 Q_value 구함
- 이 Q_value를 이용해 학습에 필요한 손실 함수를 만듬
- DQN 핵심인 목표 신경망을 학습 신경망의 값으로 갱신
- Q_vlaue를 이용해 다음 행동을 판단

## 학습 함수

### 1) 메모리에서 값 가져오기
- sample_memory 함수를 사용해 게임 플레이를 저장한 메모리에서 배치 크기만큼을 샘플링하여 가져옴

### 2) target_Q_value 구하기
- 가져온 메모리에서 다음 상태를 만들어 목표 신경망에 넣어 target_Q_value를 구함
- 현재 상태가 아닌, 다음 상태를 넣는다!!

### 3) 손실 함수에 보상값 입력
- 앞서 만든 손실 함수에 보상값을 입력
- 게임이 종료된 상태 -> 보상값을 바로 입력
- 게임이 진행중인 상태 -> 보상값에 target_Q_value 최댓값을 추가
  (현재 상황에서의 최대가치를 목표)
  
### 4) 최적화 함수 실행
- AdamOptimizer 이용한 최적화 함수에 게임 플레이 메모리에서 가져온 값들과 앞에서 구한 Y 값을 넣어 학습!

#### 메모리에서 가져온 값
- 현재 상태값
- 취한 행동  

- 몇가지 헬퍼 함수

#### 헬퍼 함수
- init_state: 학습에 사용할 상태값을 만듬
- remember: 메모리에 저장하는 함수
- _sample_memory: 추출해오는 함수

## 현재 상태 초기화 함수 init_state
- DQN에서 입력값으로 사용할 상태 = 게임판의 현재 상태 + 앞의 상태 몇 개를 합친 것
- 이를 입력값으로 만들기 위해 STATE_LEN 크기만큼의 스택으로 만들어 둠

### np.stack(state, axis=2)
- 스택을 만들면서 axis=2 옵션을 준 것은 input_X를 넣을 플레이스 홀더를 [None, width, height, self.STATE_LEN] 구성했음
- 즉, 상태들을 마지막 차원으로 쌓아올린 형태로 만들었기 때문
- 컨볼루션 계층을 손쉽게 이용

## 게임 플레이 결과를 메모리에 기억시키는 함수
### remember 함수
- 게임 플레이 결과를 받아 메모리에 기억하는 기능을 수행
- 가장 오래된 상태를 제거
- 새로운 상태를 넣어 다음 상태로 만들어 둠
- 입력받은 새로운 상태가 DQN으로 취한 행동을 통해 만들어진 상태이므로 실제로는 다음상태라고 볼 수 있다.

### 메모리에 저장하는 값들
- 게임의 현재 상태
- 게임의 다음 상태
- 취한 행동
- 그 행동으로 얻어진 보상
- 게임 종료 여부

### 저장할 플레이 개수 제한 
- 너무 오래된 과거 -> 메모리 부족 및 효율적이지 않음

## 임의의 메모리를 가져오는 함수
### _sample_memory 함수
- 기억해둔 게임 플레이에서 임의의 메모리를 배치 크기만큼 가져옴
- random.smaple 함수를 통해 임의의 메모리를 가져옴
- 현재 상태값, 다음 상태값, 취한 행동, 보상, 게임종료
- 다음 사용 쉽도록 재구성

In [ ]:
class DQN:
    
    # 하이퍼 파라미터
    REPLAY_MEMORY = 10000
    BATCH_SIZE = 32
    GAMMA = 0.99
    STATE_LEN = 4
    
    # DQN 객체 초기화
    def __init__(self, session, width, height, n_action):
        self.session = session
        self.n_action = n_action
        self.width = width
        self.height = height
        self.memory = deque()
        self.state = None
        
        # 플레이스홀더 설정
        tf.disable_eager_execution()
        
        self.input_X = tf.placeholder(tf.float32, [None, width, height, self.STATE_LEN])
        self.input_A = tf.placeholder(tf.int64, [None])
        self.input_Y = tf.placeholder(tf.float32, [None])
        
        # 신경망 구성
        self.Q = self._build_network('main')
        self.cost, self.train_op = self._build_op()
        
        self.target_Q = self.build_network('target')
        
    # 신경망 구성 함수 정의
    def _build_network(self, name):
        with tf.variable_scope(name):
            model = tf.layers.conv2d(self.input_X, 32, [4, 4], padding='same', activation=tf.nn.relu)
            model = tf.layers.conv2d(model, 64, [2, 2], padding='same', activation=tf.nn.relu)
            model = tf.contrib.layers.flatten(model)
            model = tf.layers.dense(model, 512, activation=tf.nn.relu)
            
            Q = tf.layers.dense(model, self.n_action, activation=None)
            
        return Q
    
    # DQN 손실 함수 정의
    def _build_op(self):
        one_hot = tf.one_hot(self.input_A, self.n_action, 1.0, 0.0) #indices, depth, on, off
        Q_value = tf.reduce_sum(tf.multiply(self.Q, one_hot), axis=1) # Q 값중 살아남은 값
        cost = tf.reduce_mean(tf.square(self.input_Y - Q_value))
        train_op = tf.train.AdamOptimizer(1e-6).minimize(cost)
        
        return cost, train_op
    
    # 목표 신경망 갱신 함수
    def update_target_network(self):
        copy_op = []
        
        main_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='main')
        target_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='target')
        
        for main_var, target_var in zip(main_vars, target_vars):
            copy_op.append(target_var,assign(main_var.value()))
        
        self.session.run(copy_op)
    
    # 다음 행동 정의 함수   
    def get_action(self):
        Q_value  = self.session.run(self.Q, feed_dict={self.input_X: [self.state]})
        
        action = np.argmax(Q_value[0])
        
        return action
    
    # 현재 상태 초기화 함수
    def init_state(self, state):
        state = [state for _ in range(self.STATE_LEN)]
        self.state = np.stack(state, axis=2)
    
    # 게임 플레이 결과를 메모리에 기억시키는 함수
    def remember(self, state, action, reward, terminal):
        next_state = np.reshape(state, (self,width, self,height, 1))
        next_state = np.append(self.state[:, :, 1:], next_state, axis=2)
        
        self.memory.append((self.state, next_state, action, reward, terminal))
        
        if len(self.memory) > self.REPLAY_MEMORY:
            self.memory.popleft()
        
        self.state = next_state
    
    # 임의의 메모리 가져오는 함수
    def _sample_memory(self):
        sample_memory = random.sample(self.memory, self.BATCH_SIZE)
        
        state = [memory[0] for memory in sample_memory]
        next_state = [memory[1] for memory in sample_memory]
        action = [memory[2] for memory in sample_memory]
        reward = [memory[3] for memory in sample_memory]
        terminal = [memory[4] for memory in sample_memory]
        
        return state, next_state, action, reward, terminal
    
    # 학습 함수
    def train(self):
        # 1) 메모리에서 값 가져오기
        state, next_state, action, reward, terminal = self._sample_memory()
        
        # 2) target_Q_value 구하기
        target_Q_value = self.session.run(self.target_Q, feed_dict={self.input_X: next_state})
        
        # 3) 손실 함수에 보상값 입력
        Y = []
        
        for i in range(self.BATCH_SIZE):
            if terminal[i]:
                Y.append(reward[i])
            else:
                Y.append(reward[i] + self.GAMMA * np.max(target_Q_value[i]))
        
        # 4) 최적화 함수 실행
        self.session.run(self.train_op, feed_dict={self.input_X: state, self.input_A: action, self.input_Y: Y})

In [ ]:
tf.layers.conv2d

In [57]:
tf.one_hot?

In [59]:
tf.reduce_sum?
x = tf.constant([[1, 2, 3], [1, 2, 3],[1, 2, 3], [1, 2, 3]])
tf.reduce_sum(x, 1)

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([6, 6, 6, 6], dtype=int32)>

In [8]:
tf.get_collection?